In [1]:
import os
os.chdir("../../../")

In [2]:
import pandas as pd
import re

In [3]:
input_file_path = r"D:\Studying\UoR\1. Data Mining\Final_Project\data\llama2_finetune_data\ap_canada.csv"

In [4]:
df = pd.read_csv(input_file_path)
df.shape

(10, 2)

In [5]:
df.columns

Index(['text', 'answer'], dtype='object')

# Get Class Name

In [8]:
def get_sentiment_class(text):
    pattern = "Sentiment:.+\n"
    match = re.search(pattern, text)
    
    if match is None:
        return None
    
    return text[match.start():match.end()].split()[-1]
    

get_sentiment_class(df.answer.iloc[0])

'Neutral'

In [9]:
df["sentiment_class"] = df.answer.map(get_sentiment_class)
df.sentiment_class.unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [10]:
def get_stance_class(text):
    pattern = "Stance:.+\n"
    match = re.search(pattern, text)
    
    if match is None:
        return None
    
    return text[match.start():match.end()].split()[-1]

get_stance_class(df.answer.iloc[0])

'Impartial'

In [11]:
df["stance_class"] = df.answer.map(get_sentiment_class)
df.stance_class.unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

# Get Score

In [21]:
def get_sentiment_score(text):
    pattern = "Score:.+\n"
    matches = [match for match in re.finditer(pattern, text)]
    
    if matches is None:
        return None
    
    return text[matches[0].start() : matches[0].end()].split()[-1]

get_sentiment_score(df.answer.iloc[0])

'0.0'

In [26]:
df["sentiment_score"] = df.answer.map(get_sentiment_score)
df.sentiment_score.describe()

count      10
unique      7
top       0.0
freq        3
Name: sentiment_score, dtype: object

In [24]:
def get_stance_score(text):
    pattern = "Score:.+\n"
    matches = [match for match in re.finditer(pattern, text)]
    
    if matches is None:
        return None
    
    return text[matches[1].start() : matches[1].end()].split()[-1]

get_stance_score(df.answer.iloc[0])

'0.0'

In [27]:
df["stance_score"] = df.answer.map(get_stance_score)
df.stance_score.describe()

count      10
unique      5
top       0.0
freq        4
Name: stance_score, dtype: object

# Get Reason

In [39]:
def get_sentiment_reason(text):
    pattern = "(?<=Reason:).+"
    matches = [match for match in re.finditer(pattern, text)]
    
    if matches is None:
        return None
    
    return text[matches[0].start() : matches[0].end()].strip()

get_sentiment_reason(df.answer.iloc[0])

'The article presents facts about the performance of both Yuka Saso and Brooke Henderson without expressing positive or negative emotions.'

In [33]:
df["sentiment_reason"] = df.answer.map(get_sentiment_reason)

In [40]:
def get_stance_reason(text):
    pattern = "(?<=Reason:).+"
    matches = [match for match in re.finditer(pattern, text)]
    
    if matches is None:
        return None
    
    return text[matches[1].start() : matches[1].end()].strip()

get_stance_reason(df.answer.iloc[0])

"The article neutrally reports on the event, without showing a bias towards or against Canada. It discusses both a Japanese player's success and a Canadian player's struggle."

In [41]:
df["stance_reason"] = df.answer.map(get_stance_reason)

In [44]:
df.sample(5)

,text,answer,sentiment_class,stance_class,stance_score,sentiment_score,sentiment_reason,stance_reason
2,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Positive\n * Score: 0.8\n ...,Positive,Positive,0.6,0.8,The article highlights Canada's victory and qu...,The article focuses on Canada's performance an...
9,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Positive\n * Score: 0.7\n ...,Positive,Positive,0.6,0.7,The article discusses the successful negotiati...,The article focuses on the actions of the Cana...
1,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Neutral\n * Score: 0.0\n *...,Neutral,Neutral,0.0,0.0,The article objectively reports the events of ...,"The article neutrally reports on the match, wi..."
0,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Neutral\n * Score: 0.0\n *...,Neutral,Neutral,0.0,0.0,The article presents facts about the performan...,"The article neutrally reports on the event, wi..."
4,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Neutral\n * Score: 0.0\n *...,Neutral,Neutral,0.0,0.0,The article presents both positive (appointmen...,The article neutrally reports on the changes i...


In [6]:
from src.llama2.parse_csv import ParseCSV

In [7]:
parser = ParseCSV()

In [8]:
df = parser(input_file_path)

In [9]:
df.head()

,text,answer,sentiment_class,stance_class,sentiment_score,stance_score,sentiment_reason,stance_reason
0,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Neutral\n * Score: 0.0\n *...,Neutral,Impartial,0.0,0.0,The article presents facts about the performan...,"The article neutrally reports on the event, wi..."
1,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Neutral\n * Score: 0.0\n *...,Neutral,Impartial,0.0,0.0,The article objectively reports the events of ...,"The article neutrally reports on the match, wi..."
2,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Positive\n * Score: 0.8\n ...,Positive,Pro-Canada,0.8,0.6,The article highlights Canada's victory and qu...,The article focuses on Canada's performance an...
3,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Negative\n * Score: -0.8\n ...,Negative,Against-Canada,-0.8,-0.6,The article discusses a tragic event and lawsu...,"The article implicates a Canadian company, De ..."
4,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Neutral\n * Score: 0.0\n *...,Neutral,Impartial,0.0,0.0,The article presents both positive (appointmen...,The article neutrally reports on the changes i...
